<a href="https://colab.research.google.com/github/Bidisha314/TTS_application/blob/main/Text_to_Speech_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **How to build a FREE Text-to-Speech AI App on Google Colab using Pinggy**



* Google Coab: https://colab.research.google.com/  
* TTS Model: [Kokoro-v1.onnx](https://github.com/vpakarinen/kokorotts-webui)
*  SSH Tunneling Tool: [Pinggy.io](https://pinggy.io/)



## Step 1: Git Clone Kokoro


Github link of Kokoro:  https://github.com/vpakarinen/kokoro-tts-webui.git




In [1]:
!git clone https://github.com/vpakarinen/kokoro-tts-webui.git

!cd kokoro-tts-webui

Cloning into 'kokoro-tts-webui'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 73 (delta 35), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (73/73), 102.96 KiB | 257.00 KiB/s, done.
Resolving deltas: 100% (35/35), done.


## Step 2: Create Virtual Environment

In [2]:
!rm -rf kokoro-env
!python3 -m venv kokoro-env --without-pip

## Step 3: Activate Kokoro Environment

In [4]:
!source kokoro-env/bin/activate

## Step 4: Set up Pinggy Tunnel

### Install Pinggy

In [5]:
!pip install pinggy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 79.3 MB/s eta 0:00:00


### Create Pinggy Tunnel

In [6]:
import pinggy

tunnel1 = pinggy.start_tunnel(
    forwardto="localhost:8000",
)

print(f"Tunnel1 started - URLs: {tunnel1.urls}")


Tunnel1 started - URLs: ['http://bjecq-34-187-244-175.a.free.pinggy.link', 'https://bjecq-34-187-244-175.a.free.pinggy.link']


## Step 5: Build Text to Speech Application- Twelve Labs

In [7]:
!pip install kokoro-onnx gradio soundfile torch numpy

import soundfile as sf
import urllib.request
import gradio as gr
import tempfile
import uuid
import os
from kokoro_onnx import Kokoro

# ---------- DOWNLOAD MODELS ----------
MODEL_URL = "https://github.com/thewh1teagle/kokoro-onnx/releases/download/model-files-v1.0/"
model_path = "kokoro-v1.0.onnx"
voices_bin_path = "voices-v1.0.bin"

if not os.path.exists(model_path):
    urllib.request.urlretrieve(MODEL_URL + "kokoro-v1.0.onnx", model_path)

if not os.path.exists(voices_bin_path):
    urllib.request.urlretrieve(MODEL_URL + "voices-v1.0.bin", voices_bin_path)

# ---------- LOAD MODEL ----------
kokoro = Kokoro(model_path, voices_bin_path)

# ---------- VOICES ----------
if hasattr(kokoro, "voices"):
    voice_options = list(kokoro.voices.keys())
else:
    voice_options = [
        "af_sarah", "en_erin", "en_daniel", "en_vicki",
        "en_brandon", "ja_akira", "ja_naomi", "de_anna",
        "fr_elise", "es_carlos", "it_marco", "zh_mei",
        "ru_ivan", "ko_mina"
    ]

VOICE_LABELS = {v.replace("_", " ").title(): v for v in voice_options}
# ---------- LANGUAGES ----------
LANG_LABELS = {
    "English (US)": "en-us",
    "English (UK)": "en-gb",
    "Japanese": "ja-jp",
    "Chinese (Mandarin)": "zh-cn",
    "German": "de-de",
    "Spanish": "es-es",
    "French": "fr-fr",
    "Italian": "it-it",
    "Korean": "ko-kr",
    "Portuguese (Brazil)": "pt-br",
    "Russian": "ru-ru",
}

# ---------- TTS FUNCTION ----------
def tts_generate(text, voice_label, speed, language):
    if not text.strip():
        return None, "Please enter some text."

    voice_id = VOICE_LABELS[voice_label]
    lang_code = LANG_LABELS[language]

    try:
        samples, sr = kokoro.create(
            text=text,
            voice=voice_id,
            speed=float(speed),
            lang=lang_code
        )
    except Exception as e:
        return None, f"❌ Error: {e}"

    filename = f"tts_{uuid.uuid4().hex[:8]}.wav"
    path = os.path.join(tempfile.gettempdir(), filename)
    sf.write(path, samples, sr)

    return path, "Voice generated successfully!"

# ---------- UI ----------
def build_ui():
    theme_css = """
    .gradio-container {
        max-width: 1200px !important;
        margin: 24px auto !important;
        background: #f5f7f5;
    }

    .title {
        width: 100%;
        text-align: center !important;
        font-size: 2.4rem;
        font-weight: 800;
        padding: 10px 0 5px 0;
        color: #1b3c1a;
    }

    .subtitle {
        text-align: center;
        font-size: 1.1rem;
        color: #3b4b3b;
        margin-bottom: 20px;
    }

    .card {
        padding: 28px;
        border-radius: 18px;
        background: #ffffff;
        box-shadow: 0 6px 18px rgba(0,0,0,0.08);
        overflow: visible !important;   /* FIX DROPDOWN CLIPPING */
    }

    /* DROPDOWN ALWAYS OPENS DOWN */
    .gradio-dropdown .options {
        position: absolute !important;
        top: 100% !important;
        transform: none !important;
        margin-top: 6px !important;

        background: white !important;
        border-radius: 10px !important;
        border: 1px solid #ccc !important;

        max-height: 240px !important;
        overflow-y: auto !important;

        box-shadow: 0 8px 20px rgba(0,0,0,0.15) !important;
        z-index: 99999 !important;
    }

    .green-btn button {
        background: #1ea34a !important;
        color: white !important;
        font-size: 1.18rem !important;
        padding: 16px 26px !important;
        border-radius: 12px !important;
        font-weight: 700 !important;
        height: 64px !important;
        box-shadow: 0 6px 18px rgba(0,0,0,0.14) !important;
        transition: 0.2s ease;
    }

    .green-btn button:hover {
        background: #148a3d !important;
        transform: scale(1.03);
    }
    """

    with gr.Blocks(css=theme_css, title="Twelve Labs") as app:

        gr.HTML("<div class='title'>Twelve Labs</div>")
        gr.HTML("<div class='subtitle'>Generate voice for your thoughts</div>")

        with gr.Column(elem_classes="card"):
            text_input = gr.Textbox(
                label="Enter Text",
                placeholder="Type your text here...",
                lines=4,
            )

            with gr.Row():
                voice_dropdown = gr.Dropdown(
                    label="Voice",
                    choices=list(VOICE_LABELS.keys()),
                    value=list(VOICE_LABELS.keys())[0],
                )

                lang_dropdown = gr.Dropdown(
                    label="Language",
                    choices=list(LANG_LABELS.keys()),
                    value="English (US)",
                )

            speed_slider = gr.Slider(
                0.5, 2.0, 1.0, 0.1, label="Speed"
            )

            generate_btn = gr.Button(
                "Generate Speech",
                elem_classes="green-btn"
            )

        audio_out = gr.Audio(label="Output Audio")
        status_out = gr.Markdown()

        generate_btn.click(
            tts_generate,
            inputs=[text_input, voice_dropdown, speed_slider, lang_dropdown],
            outputs=[audio_out, status_out]
        )

    return app


if __name__ == "__main__":
    app = build_ui()
    app.launch(server_name="0.0.0.0", server_port=5000, share=False)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 115.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.4/213.4 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.2/587.2 kB 50.7 MB/s eta 0:00:00


/tmp/ipython-input-2515976130.py:145: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=theme_css, title="Twelve Labs") as app:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>